In [1]:
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn import random_projection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import homogeneity_score
from sklearn.mixture import GaussianMixture
from collections import Counter
import pandas as pd
import numpy as np
import random
from pytictoc import TicToc
t = TicToc()

In [2]:
vocabulary = pd.read_csv("vocab.nytimes.csv",header=None,names=['ID','WORD'] )
vocabulary.shape

(102660, 2)

In [3]:
for ite in range(44442,vocabulary.shape[0]):
    aux_word = vocabulary["WORD"][ite]
    vocabulary["WORD"][ite] = aux_word[4:]

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_12224\417027552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocabulary["WORD"][ite] = aux_word[4:]


In [4]:
#vocabulary = vocabulary[~vocabulary['WORD'].str.contains('_', na=False, regex=False)]

In [5]:
vocabulary.shape

(102660, 2)

In [6]:
corpus = pd.read_csv("docword.nytimes.txt",header=None,names=['ARTICLE','WORD','OBSERVATIONS'],sep=' ' )

In [7]:
random.seed(19830526)
corpus.shape

(69679427, 3)

In [8]:
basePoliticsWords = ['republican','democratic','senate','congress','senator','representative']
basePoliticsWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(basePoliticsWords)].values.tolist()
articlesPolitics = corpus['ARTICLE'][corpus['WORD'].isin(basePoliticsWordsIDs)].values.tolist()
print(len(articlesPolitics))
selectedPolitics = random.sample(articlesPolitics,2000)
basePoliticsWordsIDs

123418


[7886,
 10185,
 32801,
 32842,
 35020,
 35021,
 55369,
 58011,
 88569,
 88587,
 92424,
 92479]

In [9]:
baseSportWords = ['sport','player','score','competition','championship','mvp']
baseSportWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseSportWords)].values.tolist()
articlesSports = corpus['ARTICLE'][corpus['WORD'].isin(baseSportWordsIDs)].values.tolist()
print(len(articlesSports))
selectedSports = random.sample(articlesSports,2000)
baseSportWordsIDs

115066


[6101, 7518, 25453, 29176, 34560, 37194, 53285, 55276, 81367, 91809]

In [10]:
baseNYWords = ['new_york','manhattan','queens','harlem','brooklyn','new_jersey']
baseNYWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseNYWords)].values.tolist()
articlesNY = corpus['ARTICLE'][corpus['WORD'].isin(baseNYWordsIDs)].values.tolist()
print(len(articlesNY))
selectedNY = random.sample(articlesNY,2000)
baseNYWordsIDs

90905


[4723, 17861, 23359, 51396, 66202, 77181, 82350, 82418]

In [11]:
baseClimateWords = ['climate','hurricane','storm','temperature','snow','rain']
baseClimateWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseClimateWords)].values.tolist()
articlesClimate = corpus['ARTICLE'][corpus['WORD'].isin(baseClimateWordsIDs)].values.tolist()
print(len(articlesClimate))
selectedClimate = random.sample(articlesClimate,2000)
baseClimateWordsIDs

31652


[6826, 19013, 31357, 36511, 37873, 39515, 54738, 67855, 87645, 95253]

In [12]:
baseEducationWords = ['university','graduate','student','school','teacher','learning']
baseEducationWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseEducationWords)].values.tolist()
articlesEducation = corpus['ARTICLE'][corpus['WORD'].isin(baseEducationWordsIDs)].values.tolist()
print(len(articlesEducation))
selectedEducation = random.sample(articlesEducation,2000)
baseEducationWordsIDs

141960


[17033, 22130, 34498, 38097, 39341, 41880, 65170, 95365, 96227, 99200]

In [13]:
baseTechWords = ['software','internet','data','cybersecurity','technology','digital']
baseTechWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseTechWords)].values.tolist()
articlesTech = corpus['ARTICLE'][corpus['WORD'].isin(baseTechWordsIDs)].values.tolist()
print(len(articlesTech))
selectedTech = random.sample(articlesTech,2000)
baseTechWordsIDs

90350


[9422, 9602, 10811, 20570, 36639, 39398, 68584]

In [14]:

selectedArticles = selectedTech + selectedPolitics + selectedNY + selectedClimate + selectedEducation + selectedSports
selectedWords = baseTechWordsIDs + basePoliticsWordsIDs + baseNYWordsIDs + baseClimateWordsIDs + baseEducationWordsIDs + baseSportWordsIDs
print(len(selectedArticles))
#selectedArticles = list(set(selectedArticles))
#print(len(selectedArticles))
eliminating=Counter(selectedArticles)
selectedArticlesSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedArticlesSplit.append(ite)
print(len(selectedArticlesSplit))
corpus = corpus[corpus['ARTICLE'].isin(selectedArticles)]
#words_to_keep = vocabulary['ID'].tolist()
#corpus = corpus[corpus['WORD'].isin(words_to_keep)]
corpus.shape
              

12000
11413


(3216090, 3)

In [15]:
corpus = corpus.pivot(index='ARTICLE',columns='WORD',values='OBSERVATIONS')
corpus = corpus.fillna(0)
selectedWords = selectedWords + random.sample(corpus.columns.values.tolist(),10000)    
eliminating=Counter(selectedWords)
selectedWordsSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedWordsSplit.append(ite)

In [16]:
print(corpus.shape)
#corpus = corpus.loc[:, (corpus != 0).any(axis=0)]
print(len(selectedWordsSplit))
corpus = corpus[selectedWordsSplit]
print(corpus.shape)

(11702, 80879)
10039
(11702, 10039)


In [17]:

label_real = []
for row in corpus.index:
    if row in selectedPolitics:
        label_real.append(0)
    elif row in selectedTech:
        label_real.append(1)
    elif row in selectedEducation:
        label_real.append(2)
    elif row in selectedNY:
        label_real.append(3)
    elif row in selectedClimate:
        label_real.append(4)
    elif row in selectedSports:
        label_real.append(5)

In [18]:
homogeneity_sol = pd.DataFrame([['Test',0,0,0.000,0.000]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,Time
0,Test,0,0,0.0,0.0


In [19]:
corpus = corpus.fillna(0)
corpus[corpus > 1] = 1
#columnas = list(corpus.columns)
#columnas_words = vocabulary["ID"]=

    

In [20]:
print(corpus.shape)
print(len(label_real))

(11702, 10039)
11702


In [21]:
RCA = random_projection.SparseRandomProjection()
corpus_rca = RCA.fit_transform(corpus)

In [22]:
labels_rca = []
for iteCom in range(1,50):
    t.tic()
    RCA = random_projection.SparseRandomProjection(n_components=iteCom, random_state = 19830526)
    corpus_rca = RCA.fit_transform(corpus)
    t.toc("RCA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        gm_label_rca = GaussianMixture(n_components=ite, random_state=19830526).fit_predict(corpus_rca)         
        labels_rca.append(gm_label_rca)
        taux = t.tocvalue()
        print("RCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["RCA",iteCom,ite,homogeneity_score(label_real, gm_label_rca),taux]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

RCA calibration for 1 components took  0.472377 seconds.
RCA with 1 features and 1 clusters took to run 2.2609263000000084
RCA with 1 features and 2 clusters took to run 0.08748310000000004
RCA with 1 features and 3 clusters took to run 0.028660799999997266
RCA with 1 features and 4 clusters took to run 0.02917709999999829
RCA with 1 features and 5 clusters took to run 0.04063390000000311
RCA with 1 features and 6 clusters took to run 0.04745730000000492
RCA with 1 features and 7 clusters took to run 0.06014009999999814
RCA with 1 features and 8 clusters took to run 0.06873089999999138
RCA with 1 features and 9 clusters took to run 0.09689730000000907


D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  cluster.KMeans(
D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (11). Possibly due to duplicate points in X.
  cluster.KMeans(


RCA with 1 features and 10 clusters took to run 0.08603800000000206
RCA with 1 features and 11 clusters took to run 0.09242270000000019


D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (12). Possibly due to duplicate points in X.
  cluster.KMeans(
D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (13). Possibly due to duplicate points in X.
  cluster.KMeans(


RCA with 1 features and 12 clusters took to run 0.10148460000000625
RCA with 1 features and 13 clusters took to run 0.10755089999999257


...

RCA with 20 features and 9 clusters took to run 1.351103999999964
RCA with 20 features and 10 clusters took to run 1.8927189999999996
RCA with 20 features and 11 clusters took to run 1.9309172000000103
RCA with 20 features and 12 clusters took to run 2.54557920000002
RCA with 20 features and 13 clusters took to run 2.260562200000038
RCA with 20 features and 14 clusters took to run 1.963600299999996
RCA with 20 features and 15 clusters took to run 1.8198225000000434
RCA with 20 features and 16 clusters took to run 2.302519399999994
RCA with 20 features and 17 clusters took to run 2.9902422000000115
RCA with 20 features and 18 clusters took to run 2.4236343000000033
RCA with 20 features and 19 clusters took to run 3.2184109000000376
RCA calibration for 21 components took  0.433334 seconds.
RCA with 21 features and 1 clusters took to run 0.03492190000002893
RCA with 21 features and 2 clusters took to run 0.2616852999999537
RCA with 21 features and 3 clusters took to run 0.3383964999999875

RCA with 26 features and 12 clusters took to run 2.381403999999975
RCA with 26 features and 13 clusters took to run 3.36939469999993
RCA with 26 features and 14 clusters took to run 2.2528676000000587
RCA with 26 features and 15 clusters took to run 4.719968900000026
RCA with 26 features and 16 clusters took to run 5.031156899999928
RCA with 26 features and 17 clusters took to run 2.5343359999999393
RCA with 26 features and 18 clusters took to run 3.615537199999949
RCA with 26 features and 19 clusters took to run 3.7097755000000916
RCA calibration for 27 components took  0.438253 seconds.
RCA with 27 features and 1 clusters took to run 0.04198940000003404
RCA with 27 features and 2 clusters took to run 0.2670041000000083
RCA with 27 features and 3 clusters took to run 0.4019594999999754
RCA with 27 features and 4 clusters took to run 0.8509858000001032
RCA with 27 features and 5 clusters took to run 1.1619033000000627
RCA with 27 features and 6 clusters took to run 1.749858300000028
RC

RCA with 32 features and 15 clusters took to run 3.9426381999999194
RCA with 32 features and 16 clusters took to run 5.814552100000014
RCA with 32 features and 17 clusters took to run 3.981169199999954
RCA with 32 features and 18 clusters took to run 3.681088700000032
RCA with 32 features and 19 clusters took to run 5.75598320000006
RCA calibration for 33 components took  0.458583 seconds.
RCA with 33 features and 1 clusters took to run 0.27904049999995095
RCA with 33 features and 2 clusters took to run 0.36886089999995875
RCA with 33 features and 3 clusters took to run 0.7797052999999323
RCA with 33 features and 4 clusters took to run 0.9874035999999933
RCA with 33 features and 5 clusters took to run 0.752249900000038
RCA with 33 features and 6 clusters took to run 1.3885599000000184
RCA with 33 features and 7 clusters took to run 1.5178363999999647
RCA with 33 features and 8 clusters took to run 2.1775053999999727
RCA with 33 features and 9 clusters took to run 2.6649616999999353
RCA

RCA with 38 features and 18 clusters took to run 4.062963900000113
RCA with 38 features and 19 clusters took to run 4.922253599999976
RCA calibration for 39 components took  0.464674 seconds.
RCA with 39 features and 1 clusters took to run 0.05021290000013323
RCA with 39 features and 2 clusters took to run 0.44538009999996575
RCA with 39 features and 3 clusters took to run 0.5403361999999561
RCA with 39 features and 4 clusters took to run 0.7988775000001169
RCA with 39 features and 5 clusters took to run 1.2325905999998668
RCA with 39 features and 6 clusters took to run 2.227004099999931
RCA with 39 features and 7 clusters took to run 1.480859799999962
RCA with 39 features and 8 clusters took to run 2.682679999999891
RCA with 39 features and 9 clusters took to run 3.2132222000000183
RCA with 39 features and 10 clusters took to run 2.8632816999997885
RCA with 39 features and 11 clusters took to run 3.704164500000161
RCA with 39 features and 12 clusters took to run 3.8114539000000605
RCA

RCA with 45 features and 2 clusters took to run 0.4735468999999739
RCA with 45 features and 3 clusters took to run 0.6815764999998919
RCA with 45 features and 4 clusters took to run 1.549158000000034
RCA with 45 features and 5 clusters took to run 1.3786701999999877
RCA with 45 features and 6 clusters took to run 2.0224322000001393
RCA with 45 features and 7 clusters took to run 2.283010200000035
RCA with 45 features and 8 clusters took to run 3.257627599999978
RCA with 45 features and 9 clusters took to run 3.336303199999975
RCA with 45 features and 10 clusters took to run 5.420426300000145
RCA with 45 features and 11 clusters took to run 3.4630181999998513
RCA with 45 features and 12 clusters took to run 3.304302600000028
RCA with 45 features and 13 clusters took to run 5.146674999999959
RCA with 45 features and 14 clusters took to run 4.322148900000002
RCA with 45 features and 15 clusters took to run 5.722518799999989
RCA with 45 features and 16 clusters took to run 5.76224829999978

In [23]:
labels_lda = []
   
for iteCom in range(1,6):
    t.tic()
    LDA = LinearDiscriminantAnalysis(n_components=iteCom)
    corpus_lda = LDA.fit_transform(corpus, label_real)
    t.toc("LDA calibration for "+str(iteCom)+" components took ")
    for ite in range(1,20):
        t.tic()
        gm_label_lda = GaussianMixture(n_components=ite, random_state=19830526).fit_predict(corpus_lda)         
        labels_lda.append(gm_label_lda)
        taux = t.tocvalue()
        print("LCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["LDA",iteCom,ite,homogeneity_score(label_real, gm_label_lda),taux]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

LDA calibration for 1 components took  480.453589 seconds.
LCA with 1 features and 1 clusters took to run 0.009391999999934342
LCA with 1 features and 2 clusters took to run 0.025211399999989226
LCA with 1 features and 3 clusters took to run 0.06525599999986298
LCA with 1 features and 4 clusters took to run 0.09286320000001069
LCA with 1 features and 5 clusters took to run 0.2629667999999583
LCA with 1 features and 6 clusters took to run 0.11363650000021153
LCA with 1 features and 7 clusters took to run 0.2119497000003321
LCA with 1 features and 8 clusters took to run 0.25859349999973347
LCA with 1 features and 9 clusters took to run 0.2752653000002283
LCA with 1 features and 10 clusters took to run 0.2978087000001324
LCA with 1 features and 11 clusters took to run 0.2635313999999198
LCA with 1 features and 12 clusters took to run 0.29117649999989226
LCA with 1 features and 13 clusters took to run 0.3132437999997819
LCA with 1 features and 14 clusters took to run 0.32927829999971436
LC

In [24]:
df = pd.DataFrame(label_real, index=corpus.index ,columns =['Real'])

In [25]:
homogeneity_sol.to_csv("BagOfWordsClusteringResultsEM_RCA_LDA.csv")

In [26]:
df_rca = pd.DataFrame(labels_rca, columns=corpus.index).T
df_lda = pd.DataFrame(labels_lda, columns=corpus.index).T
df_rca.to_csv("rcaEM.csv")
df_lda.to_csv("ldaEM.csv")
